In [1]:
import modelskill as ms
import mikeio1d as m1d
import pandas as pd
import numpy as np

In [2]:
# def df_column_name_generator(data, *, quantity=None, reach=None, gridpoint_index=None, chainage=None, node=None):
#     separator = ":"
#     col_head = [quantity]
#     if node is not None:
#         col_tail = [node]
#     if reach is not None:
#         if (gridpoint_index is None) and (chainage is not None):
#             col_tail = [reach, chainage]
#         elif (gridpoint_index is not None) and (chainage is None):
#             # Find chainage based on reach and index
#             raise NotImplementedError()
#         else:
#             raise ValueError("If specifying by reach, use either 'gridpoint_index' or 'chainage' but not both.")
    
#     col_elements = [str(el) for el in col_head + col_tail]
#     return separator.join(col_elements)

In [3]:
# We fabricate an observation set to try the current workflow
np.random.seed(42)

observations = pd.read_parquet('../tests/testdata/network_res1d.parquet')
observations = observations + np.random.normal(0, 10, observations.shape)
# Arbitrarily selecting 2 columns of WaterLevel that will be used as observations
relevant_columns = [col for col in observations.columns if "WaterLevel" in col][8:10]
observations = observations.loc[:, relevant_columns].rename(columns=lambda x: "sensor_" + x.split(":")[1])
observations = observations.resample("1min").mean()

## Current workflow

In [4]:
# - Loading model_results
nt = m1d.open('../tests/testdata/network.res1d')

# - Loading observations (in this cases, they are loaded from the cell above)
# observations = pd.read_csv(...)

In [5]:
obs_9 = ms.PointObservation(observations, item="sensor_9")
obs_10 = ms.PointObservation(observations, item="sensor_10")

id = "WaterLevel:7"
modres = ms.PointModelResult(nt.to_dataframe(), item=id)

comparer_1 = ms.match(obs_9, modres)
comparer_2 = ms.match(obs_10, modres)
# comparer_1 = ms.match(observations, nt.to_dataframe(), obs_item="sensor_9", mod_item=id, gtype="point")
# comparer_2 = ms.match(observations, nt.to_dataframe(), obs_item="sensor_10", mod_item=id, gtype="point")

ccol = ms.ComparerCollection([comparer_1, comparer_2])
ccol.skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
sensor_9,110,0.743778,9.167965,9.137744,7.701389,0.181837,0.047257,0.008068
sensor_10,110,0.535862,9.336934,9.321544,7.807391,0.017057,0.048156,-0.003818


The challenge is the 1d matching (?)

- [Design 1](https://github.com/DHI/modelskill/pull/536) assumes the user knows the sensor location in `mikeio1d` _coordinates_ (reach, chainage, node, catchment).
- [Design 2](https://github.com/DHI/modelskill/pull/536) suggests a more general approach where we would need to define a __shared__ coordinate system that represents a network architecture. We would need a plugin for each potential model data source (`mikeio1d`, `EPANET`, surrogates)

### Design 1

In [6]:
ms.model.point.PointType

typing.Union[str, pathlib._local.Path, pandas.core.frame.DataFrame, pandas.core.series.Series, mikeio.dfs._dfs0.Dfs0, mikeio.dataset._dataset.Dataset, mikeio.dataset._dataarray.DataArray, xarray.core.dataset.Dataset, xarray.core.dataarray.DataArray, mikeio1d.res1d.Res1D]

In [7]:
res = ms.PointModelResult(nt, item="WaterLevel:7")
obs_1 = ms.obs.NetworkPointObservation(observations, item="sensor_9")
obs_2 = ms.obs.NetworkPointObservation(observations, item="sensor_10")

comparer_1 = ms.match(obs_1, res)
comparer_2 = ms.match(obs_2, res)

ccol = ms.ComparerCollection([comparer_1, comparer_2])
ccol.skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
sensor_9,110,0.743778,9.167965,9.137744,7.701389,0.181837,0.047257,0.008068
sensor_10,110,0.535862,9.336934,9.321544,7.807391,0.017057,0.048156,-0.003818


## Sandbox

A dataframe has the number of columns equal to the number of gridpoints + nodes.

In [ ]:
def get_n_quantities(nt: m1d.result_network.ResultNetwork):
    i = 0
    for reach in nt.reaches.values():
        for n in reach.gridpoints:
            i += len(n.quantities)

    for node in nt.nodes.values():
        i += len(node.quantities)
    
    return i


n_quantities = get_n_quantities(nt)

assert n_quantities == nt.to_dataframe().shape[1]

In [73]:
quantity = "WaterLevel"
quantity_in_nodes = quantity in nt.nodes.quantities.keys()
quantity_in_reaches = nt.reaches.quantities.keys()

In [ ]:
# - Specify relevant quantity
# - Iterate over (mike1d)reaches and select gridpoints with such quantity, we will call them nodes
#   - Check if (mike1d)nodes contain such quantity
# - Fill quantity network
# - Location in the network is specified with two

In [74]:
key_map = list(nt.nodes.keys())

assert len(set(key_map)) == len(key_map)  # checking that there are no duplicate node names

n_nodes = len(key_map)
adjacency = [[]] * n_nodes
for reach in nt.reaches.values():
    i = key_map.index(reach.start_node)
    j = key_map.index(reach.end_node)
    adjacency[i].append(j)

edges_by_node = [len(ad) for ad in adjacency]
adjacency = [l for sublist in adjacency for l in sublist]

In [77]:
nt.reaches["100l1"].start_node

'100'

In [75]:
adjacency
edges_by_node

[118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118,
 118]

In [35]:
k = 3
i = sum(edges_by_node[:k]) + edges_by_node[k]
adjacency[i]

IndexError: list index out of range

In [29]:
[len(n) for n in adjacency] 

TypeError: object of type 'int' has no len()

In [ ]:
np.zeros(n_nodes)

119

In [52]:
quantity = "WaterLevel"

getattr(nt.nodes, quantity).to_dataframe()

,WaterLevel:1,WaterLevel:2,WaterLevel:3,WaterLevel:4,WaterLevel:5,WaterLevel:6,WaterLevel:7,WaterLevel:8,WaterLevel:9,WaterLevel:10,...,WaterLevel:46,WaterLevel:55,WaterLevel:58,WaterLevel:116,WaterLevel:117,WaterLevel:118,WaterLevel:115,WaterLevel:119,WaterLevel:120,WaterLevel:Weir Outlet:119w1
1994-08-07 16:35:00.000,195.052994,195.821503,195.8815,193.604996,193.615005,193.625000,193.675003,193.764999,193.774994,193.804993,...,194.074997,195.005005,193.554993,193.550003,193.585007,193.585007,193.304993,193.550003,193.550003,193.779999
1994-08-07 16:36:01.870,195.052994,195.821701,195.8815,193.604996,193.615005,193.625320,193.675110,193.765060,193.775116,193.804993,...,194.074997,195.005005,193.555023,193.550064,193.585831,193.586807,193.306061,193.550003,193.550003,188.479996
1994-08-07 16:37:07.560,195.052994,195.821640,195.8815,193.604996,193.615005,193.625671,193.675369,193.765106,193.775513,193.804993,...,194.074997,195.005005,193.555084,193.550110,193.586426,193.588196,193.307144,193.550034,193.550003,188.479996
1994-08-07 16:38:55.828,195.052994,195.821503,195.8815,193.604996,193.615005,193.626236,193.675751,193.765228,193.776077,193.804993,...,194.074997,195.005005,193.555191,193.550156,193.586960,193.589706,193.308884,193.550079,193.550003,188.479996
1994-08-07 16:39:55.828,195.052994,195.821503,195.8815,193.604996,193.615005,193.626556,193.675949,193.765335,193.776352,193.804993,...,194.074997,195.005005,193.555267,193.550171,193.587112,193.590317,193.309860,193.550095,193.550003,188.479996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994-08-07 18:30:07.967,195.119919,195.822769,195.8815,193.884308,193.931808,193.961746,193.977524,193.999054,194.010910,194.023590,...,194.087036,195.025726,193.832489,193.830322,193.755844,193.832626,193.755829,193.800369,193.550003,188.479996
1994-08-07 18:31:07.967,195.118607,195.822769,195.8815,193.879333,193.926086,193.955582,193.971191,193.992905,194.004776,194.017578,...,194.086807,195.025391,193.828842,193.826797,193.756500,193.828964,193.756470,193.798798,193.550003,188.479996
1994-08-07 18:32:07.967,195.117310,195.822769,195.8815,193.874542,193.920532,193.949615,193.965073,193.986984,193.998856,194.011795,...,194.086609,195.025070,193.825348,193.823456,193.757156,193.825485,193.757172,193.797256,193.550003,188.479996
1994-08-07 18:33:07.967,195.115753,195.822769,195.8815,193.869873,193.915131,193.943802,193.959091,193.981171,193.993011,194.006042,...,194.086411,195.024796,193.822006,193.820160,193.757736,193.822144,193.757660,193.795715,193.550003,188.479996


In [31]:
nt.reaches["100l1"]

<Reach: 100l1>

In [18]:
nt.reaches.quantities["WaterLevel"].to_dataframe()

,WaterLevel:100l1:0,WaterLevel:100l1:47.6827,WaterLevel:101l1:0,WaterLevel:101l1:66.4361,WaterLevel:102l1:0,WaterLevel:102l1:10.9366,WaterLevel:103l1:0,WaterLevel:103l1:26.0653,WaterLevel:104l1:0,WaterLevel:104l1:34.4131,...,WaterLevel:98l1:0,WaterLevel:98l1:16.0098,WaterLevel:99l1:0,WaterLevel:99l1:44.5016,WaterLevel:9l1:0,WaterLevel:9l1:10,WaterLevel:Weir:119w1:0,WaterLevel:Weir:119w1:1,WaterLevel:Pump:115p1:0,WaterLevel:Pump:115p1:82.4281
1994-08-07 16:35:00.000,195.441498,194.661499,195.931503,195.441498,193.550003,193.550003,195.801498,195.701508,197.072006,196.962006,...,194.581497,194.511505,194.661499,194.581497,193.774994,193.764999,193.550003,188.479996,193.304993,195.005005
1994-08-07 16:36:01.870,195.441498,194.661621,195.931503,195.441605,193.550140,193.550064,195.801498,195.703171,197.072006,196.962051,...,194.581497,194.511841,194.661575,194.581497,193.775070,193.765060,193.550003,188.479996,193.306061,195.005005
1994-08-07 16:37:07.560,195.441498,194.661728,195.931503,195.441620,193.550232,193.550156,195.801498,195.703400,197.072006,196.962082,...,194.581497,194.511795,194.661667,194.581497,193.775391,193.765106,193.550034,188.479996,193.307144,195.005005
1994-08-07 16:38:55.828,195.441498,194.661804,195.931503,195.441605,193.550369,193.550308,195.801498,195.703690,197.072006,196.962112,...,194.581497,194.511581,194.661865,194.581497,193.775894,193.765228,193.550079,188.479996,193.308884,195.005005
1994-08-07 16:39:55.828,195.441498,194.661972,195.931503,195.441605,193.550430,193.550369,195.801498,195.703827,197.072006,196.962128,...,194.581497,194.511505,194.661911,194.581497,193.776154,193.765335,193.550095,188.479996,193.309860,195.005005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994-08-07 18:30:07.967,195.455109,194.689072,195.931946,195.470230,193.850647,193.850388,195.801498,195.787277,197.072006,196.967056,...,194.601105,194.533127,194.685715,194.607559,194.000839,193.999054,193.800369,188.479996,193.755829,195.025726
1994-08-07 18:31:07.967,195.455063,194.688934,195.931915,195.470108,193.846222,193.845978,195.801498,195.785614,197.072006,196.967041,...,194.601013,194.532974,194.685608,194.607407,193.994766,193.992905,193.798798,188.479996,193.756470,195.025391
1994-08-07 18:32:07.967,195.455002,194.688812,195.931885,195.469986,193.841980,193.841751,195.801498,195.783936,197.072006,196.967026,...,194.600922,194.532822,194.685516,194.607254,193.988892,193.986984,193.797256,188.479996,193.757172,195.025070
1994-08-07 18:33:07.967,195.453049,194.688354,195.931870,195.467972,193.837784,193.837585,195.801498,195.782394,197.072006,196.966660,...,194.600815,194.532684,194.685074,194.607101,193.983109,193.981171,193.795715,188.479996,193.757660,195.024796


In [17]:
nt.nodes.quantities["WaterLevel"].to_dataframe()

,WaterLevel:1,WaterLevel:2,WaterLevel:3,WaterLevel:4,WaterLevel:5,WaterLevel:6,WaterLevel:7,WaterLevel:8,WaterLevel:9,WaterLevel:10,...,WaterLevel:46,WaterLevel:55,WaterLevel:58,WaterLevel:116,WaterLevel:117,WaterLevel:118,WaterLevel:115,WaterLevel:119,WaterLevel:120,WaterLevel:Weir Outlet:119w1
1994-08-07 16:35:00.000,195.052994,195.821503,195.8815,193.604996,193.615005,193.625000,193.675003,193.764999,193.774994,193.804993,...,194.074997,195.005005,193.554993,193.550003,193.585007,193.585007,193.304993,193.550003,193.550003,193.779999
1994-08-07 16:36:01.870,195.052994,195.821701,195.8815,193.604996,193.615005,193.625320,193.675110,193.765060,193.775116,193.804993,...,194.074997,195.005005,193.555023,193.550064,193.585831,193.586807,193.306061,193.550003,193.550003,188.479996
1994-08-07 16:37:07.560,195.052994,195.821640,195.8815,193.604996,193.615005,193.625671,193.675369,193.765106,193.775513,193.804993,...,194.074997,195.005005,193.555084,193.550110,193.586426,193.588196,193.307144,193.550034,193.550003,188.479996
1994-08-07 16:38:55.828,195.052994,195.821503,195.8815,193.604996,193.615005,193.626236,193.675751,193.765228,193.776077,193.804993,...,194.074997,195.005005,193.555191,193.550156,193.586960,193.589706,193.308884,193.550079,193.550003,188.479996
1994-08-07 16:39:55.828,195.052994,195.821503,195.8815,193.604996,193.615005,193.626556,193.675949,193.765335,193.776352,193.804993,...,194.074997,195.005005,193.555267,193.550171,193.587112,193.590317,193.309860,193.550095,193.550003,188.479996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994-08-07 18:30:07.967,195.119919,195.822769,195.8815,193.884308,193.931808,193.961746,193.977524,193.999054,194.010910,194.023590,...,194.087036,195.025726,193.832489,193.830322,193.755844,193.832626,193.755829,193.800369,193.550003,188.479996
1994-08-07 18:31:07.967,195.118607,195.822769,195.8815,193.879333,193.926086,193.955582,193.971191,193.992905,194.004776,194.017578,...,194.086807,195.025391,193.828842,193.826797,193.756500,193.828964,193.756470,193.798798,193.550003,188.479996
1994-08-07 18:32:07.967,195.117310,195.822769,195.8815,193.874542,193.920532,193.949615,193.965073,193.986984,193.998856,194.011795,...,194.086609,195.025070,193.825348,193.823456,193.757156,193.825485,193.757172,193.797256,193.550003,188.479996
1994-08-07 18:33:07.967,195.115753,195.822769,195.8815,193.869873,193.915131,193.943802,193.959091,193.981171,193.993011,194.006042,...,194.086411,195.024796,193.822006,193.820160,193.757736,193.822144,193.757660,193.795715,193.550003,188.479996


In [27]:
nt.reaches["105l1"]["0.000"]

In [28]:
nt.reaches["105l1"].gridpoints

[<ResultGridPoint>, <ResultGridPoint>, <ResultGridPoint>]

In [18]:
nt.reaches["102l1"]["10.937"].quantities

['WaterLevel']

In [5]:
temperature = ms.PointModelResult(datapath, item="Temperature", name="Temp")

In [3]:
ds.to_dataframe()

,Temperature,Water Level
2015-01-01 01:00:00,3.14,0.1400
2015-01-01 02:00:00,3.14,0.1370
2015-01-01 03:00:00,3.14,0.1809
2015-01-01 04:00:00,3.14,0.2030
2015-01-01 05:00:00,3.14,0.1750
...,...,...
2020-09-27 20:00:00,3.14,0.4070
2020-09-27 21:00:00,3.14,0.3790
2020-09-27 22:00:00,3.14,0.3650
2020-09-27 23:00:00,3.14,0.3790
